#Connect to drive

In [1]:
!git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi
%cd RecSys_Course_AT_PoliMi
!pip install -r requirements.txt
!python run_compile_all_cython.py

Cloning into 'RecSys_Course_AT_PoliMi'...
remote: Enumerating objects: 1499, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 1499 (delta 113), reused 231 (delta 100), pack-reused 1255
Receiving objects: 100% (1499/1499), 50.98 MiB | 20.80 MiB/s, done.
Resolving deltas: 100% (844/844), done.
/content/RecSys_Course_AT_PoliMi
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.0/978.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 kB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies ... error
error: subprocess-exited-with-error

× pip subprocess t

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


#Import Library

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from typing import Tuple, Callable, Dict, Optional, List
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from scipy.stats import loguniform
!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.5 MB/s eta 0:00:00


#Load Data

In [4]:
%cd /gdrive/MyDrive/RECSYS
data_train = pd.read_csv('data_train.csv')
data_target_users_test = pd.read_csv('data_target_users_test.csv')
%cd ../../../content/RecSys_Course_AT_PoliMi

/gdrive/MyDrive/RECSYS
/content/RecSys_Course_AT_PoliMi


In [5]:
data_train.columns = ["user_id", "item_id", "Interaction"]

In [6]:
print ("The number of interactions is {}".format(len(data_train)))

The number of interactions is 478730


In [7]:
userID_unique = data_train["user_id"].unique()
itemID_unique = data_train["item_id"].unique()

In [8]:
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(data_train)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))

Number of items	 22222, Number of users	 12638
Max ID items	 22347, Max Id users	 13024



#Splitting Data

In [9]:
def preprocess_data(interactions: pd.DataFrame):
    unique_users = interactions.user_id.unique()
    unique_items = interactions.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    interactions = pd.merge(left=interactions,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    interactions = pd.merge(left=interactions,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return interactions


In [10]:
interactions = preprocess_data(data_train)

12638 1 13024
22222 1 22347


In [11]:
interactions

,user_id,item_id,Interaction,mapped_user_id,mapped_item_id
0,1,7,1.0,0,0
1,2,7,1.0,1,0
2,26,7,1.0,24,0
3,36,7,1.0,34,0
4,41,7,1.0,39,0
...,...,...,...,...,...
478725,12962,20368,1.0,12579,22217
478726,12985,21058,1.0,12601,22218
478727,12989,22317,1.0,12605,22219
478728,13009,22339,1.0,12624,22220


In [12]:
def dataset_splits(interactions, num_users, num_items, validation_percentage: float, testing_percentage: float):
    seed = 1234

    (user_ids_training, user_ids_test,
     item_ids_training, item_ids_test,
     interactions_training, interactions_test) = train_test_split(interactions.mapped_user_id,
                                                        interactions.mapped_item_id,
                                                        interactions.Interaction,
                                                        test_size=testing_percentage,
                                                        shuffle=True,
                                                        random_state=seed)

    (user_ids_training, user_ids_validation,
     item_ids_training, item_ids_validation,
     interactions_training, interactions_validation) = train_test_split(user_ids_training,
                                                              item_ids_training,
                                                              interactions_training,
                                                              test_size=validation_percentage,
                                                             )

    urm_train = sp.csr_matrix((interactions_training, (user_ids_training, item_ids_training)),
                              shape=(num_users, num_items))

    urm_validation = sp.csr_matrix((interactions_validation, (user_ids_validation, item_ids_validation)),
                              shape=(num_users, num_items))

    urm_test = sp.csr_matrix((interactions_test, (user_ids_test, item_ids_test)),
                              shape=(num_users, num_items))



    return urm_train, urm_validation, urm_test



In [13]:
urm_train, urm_validation, urm_test = dataset_splits(interactions,
                                                     num_users=12638,
                                                     num_items=22222,
                                                     validation_percentage=0.20,
                                                     testing_percentage=0.20)

In [14]:
urm_train

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 306387 stored elements in Compressed Sparse Row format>

In [15]:
urm_validation

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 76597 stored elements in Compressed Sparse Row format>

In [16]:
urm_test

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 95746 stored elements in Compressed Sparse Row format>

#Build the Recommendation System

In [109]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [ ]:
import pandas as pd
from tqdm import tqdm
import scipy.sparse as sps
import numpy as np
from xgboost import XGBRanker

n_users, n_items = urm_train.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'

In [ ]:
cutoff = 10

for user_id in tqdm(range(n_users)):
    recommendations = itemKNNCF.recommend(user_id, cutoff = cutoff)
    training_dataframe.loc[user_id, "ItemID"] = recommendations

In [ ]:
training_dataframe = training_dataframe.explode("ItemID")
training_dataframe

In [ ]:
URM_validation_coo = sps.coo_matrix(urm_validation)

correct_recommendations = pd.DataFrame({"UserID": URM_validation_coo.row,
                                        "ItemID": URM_validation_coo.col})
correct_recommendations

In [ ]:
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
training_dataframe

In [ ]:
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)
training_dataframe

In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

best_rp3 = {'topK': 60, 'alpha': 0.33948653564351783, 'beta': 0.03959118666931211}
best_p3 = {'topK': 65, 'alpha': 0.19508730035495367, 'implicit': False, 'normalize_similarity': True}

pureSVD = PureSVDRecommender(urm_train)
pureSVD.fit()

p3alpha = P3alphaRecommender(urm_train)
p3alpha.fit(**best_p3)

rp3 = RP3betaRecommender(urm_train)
rp3.fit(**best_rp3)


other_algorithms = {
    "PureSVD": pureSVD,
    "P3alpha": p3alpha,
    "RP3beta": rp3
}

In [ ]:
training_dataframe = training_dataframe.set_index('UserID')

for user_id in tqdm(range(n_users)):
    for rec_label, rec_instance in other_algorithms.items():

        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()

        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)

        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list]

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})
training_dataframe

In [ ]:
user_popularity = np.ediff1d(sps.csr_matrix(urm_train).indptr)

training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
training_dataframe

In [ ]:
training_dataframe = training_dataframe.sort_values("UserID").reset_index()
training_dataframe.drop(columns = ['index'], inplace=True)
training_dataframe

In [ ]:
groups = training_dataframe.groupby("UserID").size().values
groups

In [ ]:
from xgboost import XGBRanker

In [ ]:
n_estimators = 50
learning_rate = 1e-1
reg_alpha = 1e-1
reg_lambda = 1e-1
max_depth = 5
max_leaves = 0
grow_policy = "depthwise"
objective = "pairwise"
booster = "gbtree"
use_user_profile = False
random_seed = None

XGB_model = XGBRanker(objective='rank:{}'.format(objective),
                      n_estimators = int(n_estimators),
                      random_state = random_seed,
                      learning_rate = learning_rate,
                      reg_alpha = reg_alpha,
                      reg_lambda = reg_lambda,
                      max_depth = int(max_depth),
                      max_leaves = int(max_leaves),
                      grow_policy = grow_policy,
                      verbosity = 0, # 2 if self.verbose else 0,
                      booster = booster,
                      )

In [ ]:
training_dataframe["Label"].value_counts()

In [ ]:

y_train = training_dataframe["Label"]
X_train = training_dataframe.drop(columns=["Label"])


In [ ]:

X_train["UserID"] = X_train["UserID"].astype("category")
X_train["ItemID"] = X_train["ItemID"].astype("category")


XGB_model = XGBRanker(objective='rank:{}'.format(objective),
                      n_estimators = int(n_estimators),
                      random_state = random_seed,
                      learning_rate = learning_rate,
                      reg_alpha = reg_alpha,
                      reg_lambda = reg_lambda,
                      max_depth = int(max_depth),
                      max_leaves = int(max_leaves),
                      grow_policy = grow_policy,
                      verbosity = 0, # 2 if self.verbose else 0,
                      booster = booster,
                      enable_categorical = True,
                      tree_method = "hist",  # Supported tree methods are `gpu_hist`, `approx`, and `hist`.
                      )

XGB_model.fit(X_train,
          y_train,
          group=groups,
          verbose=True)

In [ ]:
X_to_predict = X_train[X_train["UserID"] == 10]

XGB_model.predict(X_to_predict)

In [ ]:
%matplotlib inline
from xgboost import plot_importance
plot_importance(XGB_model, importance_type='weight', title='Weight (Frequence)')

## Top popular for the users without info

In [17]:
class TopPopRecommender(object):

    def fit(self, URM_train):

        item_popularity = np.ediff1d(URM_train.tocsc().indptr)

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = np.argsort(item_popularity)
        self.popular_items = np.flip(self.popular_items, axis = 0)


    def recommend(self, user_id, at=5):

        recommended_items = self.popular_items[0:at]

        return recommended_items

#Hyperparameter Tuning


*   Number of neighbors

*   Shrinking

*   Similarity Type



In [18]:
from Evaluation.Evaluator import EvaluatorHoldout


evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2600 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2152 (17.0%) Users that have less than 1 test interactions


In [115]:
from Recommenders.GraphBased.RP3betaRecommender import *
def objective_function(optuna_trial):
  rp3 = RP3betaRecommender(urm_train)
  rp3.fit(topK = optuna_trial.suggest_int("topK", 5,200),
          alpha = optuna_trial.suggest_float("alpha", 1e-3, 1.0, log=True),
          beta = optuna_trial.suggest_float("beta", 1e-3, 0.8, log=True),
          min_rating=optuna_trial.suggest_categorical("min_rating", [0,1]),
          implicit = True
          )
  result_df, _ = evaluator_validation.evaluateRecommender(rp3)
  return result_df.loc[10]["MAP"]

In [91]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame()

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)

In [ ]:
optuna_study = optuna.create_study(direction="maximize")

save_results = SaveResults()

optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 1000)

In [19]:
rp3_bests = {'topK': 38, 'alpha': 0.26757664834748907, 'beta': 0.11595928151320324, 'min_rating': 1,'implicit': True}

In [20]:
item_bests = {'similarity': 'tversky', 'topK': 5, 'shrink': 21, 'tversky_alpha': 0.13257209212599813, 'tversky_beta': 1.5891321967903664}

In [119]:
recommender_instance_item = ItemKNNCFRecommender(urm_train + urm_validation)
recommender_instance_item.fit(**item_bests)


ItemKNNCFRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 93 ( 0.4%) items with no interactions.
Similarity column 22222 (100.0%), 3731.00 column/sec. Elapsed time 5.96 sec


In [120]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import *
def objective_function_ItemKNN_similarities(optuna_trial):

  itemknn = ItemKNNCFRecommender(urm_train)
  itemknn.fit(**item_bests)
  rp3 = RP3betaRecommender(urm_train)
  rp3.fit(**rp3_bests)

  hybrid = ItemKNNSimilarityHybridRecommender(urm_train, itemknn.W_sparse, rp3.W_sparse)

  hybrid.fit(topK = optuna_trial.suggest_int("topK", 5,500),
      alpha = optuna_trial.suggest_float("alpha", 0.01, 1.0, log=True))
  result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
  return result_df.loc[10]["MAP"]

In [ ]:
optuna_study = optuna.create_study(direction="maximize")

save_results = SaveResults()

optuna_study.optimize(objective_function_ItemKNN_similarities,
                      callbacks=[save_results],
                      n_trials = 100)

[I 2023-11-28 17:22:37,914] A new study created in memory with name: no-name-e2e7e186-2eb7-4ce4-af16-c9cfff912c3d


ItemKNNCFRecommender: URM Detected 490 ( 3.9%) users with no interactions.
ItemKNNCFRecommender: URM Detected 311 ( 1.4%) items with no interactions.
Similarity column 22222 (100.0%), 3418.50 column/sec. Elapsed time 6.50 sec
RP3betaRecommender: URM Detected 490 ( 3.9%) users with no interactions.
RP3betaRecommender: URM Detected 311 ( 1.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3014.94 column/sec. Elapsed time 7.37 sec
ItemKNNSimilarityHybridRecommender: URM Detected 490 ( 3.9%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 311 ( 1.4%) items with no interactions.
EvaluatorHoldout: Processed 10039 (100.0%) in 7.93 sec. Users per second: 1266


[I 2023-11-28 17:23:02,170] Trial 0 finished with value: 0.03096895915453568 and parameters: {'topK': 329, 'alpha': 0.0108767852018699}. Best is trial 0 with value: 0.03096895915453568.


ItemKNNCFRecommender: URM Detected 490 ( 3.9%) users with no interactions.
ItemKNNCFRecommender: URM Detected 311 ( 1.4%) items with no interactions.


<ipython-input-91-14c86811ef81>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


...

<ipython-input-91-14c86811ef81>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


Similarity column 22222 (100.0%), 3034.55 column/sec. Elapsed time 7.32 sec
RP3betaRecommender: URM Detected 490 ( 3.9%) users with no interactions.
RP3betaRecommender: URM Detected 311 ( 1.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3251.77 column/sec. Elapsed time 6.83 sec
ItemKNNSimilarityHybridRecommender: URM Detected 490 ( 3.9%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 311 ( 1.4%) items with no interactions.
EvaluatorHoldout: Processed 10039 (100.0%) in 8.31 sec. Users per second: 1208


[I 2023-11-28 17:55:02,993] Trial 82 finished with value: 0.03170048714774305 and parameters: {'topK': 95, 'alpha': 0.25683918021579116}. Best is trial 63 with value: 0.031730528715786306.


ItemKNNCFRecommender: URM Detected 490 ( 3.9%) users with no interactions.
ItemKNNCFRecommender: URM Detected 311 ( 1.4%) items with no interactions.


<ipython-input-91-14c86811ef81>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


In [23]:
itemknn = ItemKNNCFRecommender(urm_train+urm_validation)
itemknn.fit(**item_bests)
from Recommenders.GraphBased.RP3betaRecommender import *
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import *
rp3 = RP3betaRecommender(urm_train + urm_validation)
rp3.fit(**rp3_bests)

best_hybrid = {'topK': 62, 'alpha': 0.34054720087980433}

scoreshybridrecommender = ItemKNNSimilarityHybridRecommender(urm_train + urm_validation, itemknn.W_sparse, rp3.W_sparse)
scoreshybridrecommender.fit(**best_hybrid)

result_df, _ = evaluator_test.evaluateRecommender(scoreshybridrecommender)
result_df

ItemKNNCFRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 93 ( 0.4%) items with no interactions.
Similarity column 22222 (100.0%), 4249.25 column/sec. Elapsed time 5.23 sec
RP3betaRecommender: URM Detected 230 ( 1.8%) users with no interactions.
RP3betaRecommender: URM Detected 93 ( 0.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2288.58 column/sec. Elapsed time 9.71 sec
ItemKNNSimilarityHybridRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 93 ( 0.4%) items with no interactions.
EvaluatorHoldout: Processed 10486 (100.0%) in 6.68 sec. Users per second: 1570


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.095861,0.163302,0.136037,0.050169,0.085337,0.273324,0.15162,0.112469,0.511921,0.364946,...,0.82972,0.424751,0.82972,0.103121,10.956698,0.998788,0.296058,0.844088,1.25308,0.303579


#Prepare Submission

In [27]:
urm_train_validation_test = urm_train + urm_validation + urm_test
urm_train_validation_test

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [25]:
itemknn = ItemKNNCFRecommender(urm_train+urm_validation +urm_test)
itemknn.fit(**item_bests)
rp3 = RP3betaRecommender(urm_train + urm_validation +urm_test)
rp3.fit(**rp3_bests)

best_recommender = ItemKNNSimilarityHybridRecommender(urm_train_validation_test, itemknn.W_sparse, rp3.W_sparse)
best_recommender.fit(**best_hybrid)

Similarity column 22222 (100.0%), 2853.97 column/sec. Elapsed time 7.79 sec
RP3betaRecommender: Similarity column 22222 (100.0%), 1178.10 column/sec. Elapsed time 18.86 sec


##Top popular for those users without info

In [28]:
toppop_recommender = TopPopRecommender()
toppop_recommender.fit(urm_train_validation_test)

In [29]:
users_to_recommend = np.array(data_target_users_test["user_id"])
len(users_to_recommend)

10882

In [30]:
def prepare_submission(interactions: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object, toppop_recommender: object):
    users_ids_and_mappings = interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates()
    items_ids_and_mappings = interactions[["item_id", "mapped_item_id"]].drop_duplicates()

    mapping_to_item_id = dict(zip(interactions.mapped_item_id, interactions.item_id))


    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id

        recommendations = recommender.recommend(user_id_array=mapped_user_id, cutoff=10)

        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    #assign top popular to the user without interactions
    users_without_info = np.setdiff1d(users_to_recommend,np.array(interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates().user_id))
    for user_id in users_without_info:
      recommendations = toppop_recommender.recommend(user_id, at = 10)
      submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    return submission

In [31]:
submission = prepare_submission(interactions, users_to_recommend, urm_train_validation_test, best_recommender, toppop_recommender)

In [32]:
submission

[(1, [36, 101, 506, 123, 403, 694, 592, 515, 342, 618]),
 (2, [1095, 47, 28, 50, 1522, 102, 12, 656, 949, 380]),
 (26, [41, 254, 55, 1788, 139, 269, 252, 15, 34, 314]),
 (36, [181, 16, 2205, 1578, 4, 3870, 1825, 202, 2791, 1393]),
 (41, [31, 58, 55, 77, 66, 60, 618, 175, 605, 592]),
 (47, [96, 214, 38, 344, 211, 1635, 442, 85, 262, 154]),
 (54, [2, 19, 238, 8, 174, 278, 157, 6, 769, 112]),
 (73, [516, 125, 998, 21, 580, 1728, 2660, 10057, 1112, 701]),
 (88, [346, 99, 11531, 1222, 1089, 2725, 133, 139, 254, 77]),
 (89, [138, 324, 154, 344, 433, 186, 12673, 59, 12672, 15014]),
 (95, [482, 44, 2456, 2178, 1842, 18638, 8903, 12504, 2933, 10884]),
 (100, [313, 764, 99, 636, 1050, 11, 17, 851, 262, 50]),
 (101, [2314, 837, 82, 526, 1177, 754, 624, 1207, 137, 978]),
 (102, [7055, 5724, 2904, 2958, 2284, 67, 1795, 5725, 7030, 2843]),
 (104, [119, 299, 252, 23, 639, 75, 2, 89, 129, 71]),
 (114, [17, 5, 575, 50, 293, 5427, 186, 37, 27, 7236]),
 (124, [346, 9878, 3520, 1342, 715, 859, 1198, 3954,

In [33]:
def write_submission(submissions):
    with open("./submission.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [34]:
%cd /gdrive/MyDrive/RECSYS
write_submission(submission)

/gdrive/MyDrive/RECSYS
